# Coletor de dados da busca do site RunRepeat

<div>
<img src="imagens/rr.png" align="left" />
</div>

<a href="https://runrepeat.com/" target="_blank">RunRepeat Official Website</a>

---
### Rodrigo Fragoso 
- [**Linkedin**](https://www.linkedin.com/in/rodrigo-a-fragoso/) <br/>
- **Email** : rodrigoandradefragoso@gmail.com <br/>

### Objetivo
-  Nesta etapa, iremos coletar os dados disponíveis na página de busca para a categoria *running shoes* do site RunRepeat;
-  Os outputs esperados são: o link da página de cada tênis disponível e, ocasionalmente, alguma informação adicional que possa ser aproveitada posteriormente.
---

<a id='top'></a>
## Sumário

* [1 - Importações das bibliotecas](#t1)

* [2 - Coleta dos dados da página de busca](#t2)

* [3 - Processamento dos dados brutos](#t3)

* [4 - Verificação do resultado](#t4)


##     

<a id='t1'></a>
## 1 - Importações das bibliotecas
- [Sumário](#top)   
    - [Próximo](#t2)

### Para iniciarmos a extração será necessário o uso de algumas bibliotecas específicas que serão importadas na célula abaixo:
-  Pandas: ferramenta rápida e poderosa, responsável pela manipulação/analise de dados através do formato *dataframe* ;
-  re: modulo para realizar operações de correspondência (em texto) através de expressões regulares ;
- time: modulo utilizado, principalmente, para cálculo de tempo de processamento e criação de *delays* ;
- requests:  biblioteca HTTP utilizada para fazer o download do código fonte da página ;
- bs4 (Beautiful Soup 4): biblioteca utilizada para extrair dados de arquivos HTML e XML, utilizada como *parser* para navegarmos dentro dos arquivos criados pela requests.

In [1]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4 
import json

##     

<a id='t2'></a>
## 2 - Coleta dos dados da página de busca
- [Sumário](#top) 
    - [Anterior](#t1)
    - [Próximo](#t3)

### Nesta etapa, iremos navegar pela página de busca da classe *running shoes* e coletar as informações disponíveis em cada uma delas.
- Primeiramente, são criadas algumas variáveis para auxiliar na organização dos dados que serão extraídos:

In [2]:
url = "https://runrepeat.com/ranking/rankings-of-running-shoes?page={page}"
pagina='pagina'

- Na data em que o site foi acessado, percebemos que existiam 76 páginas na categoria desejada (em python, o limite superior do *for* não é utilizado, por isso temos o range até 77). Sendo assim, com o auxílio da biblioteca requests, os dados brutos foram salvos, um arquivo por página, no formato html ;
- O delay de dois segundos foi inserido para evitar que o nosso acesso fosse bloqueado pela segurança do site ao fazer muitas requisições em um intervalo de tempo pequeno.

In [3]:
for page in range(1,77):
    urll = url.format(page=page)
    print(urll)
    response = rq.get(urll)

    with open("./dados_brutos/{}_{}.html".format(pagina,page), 'w+',encoding="utf-8") as output:
        output.write(response.text)
    time.sleep(2)

https://runrepeat.com/ranking/rankings-of-running-shoes?page=1
https://runrepeat.com/ranking/rankings-of-running-shoes?page=2
https://runrepeat.com/ranking/rankings-of-running-shoes?page=3
https://runrepeat.com/ranking/rankings-of-running-shoes?page=4
https://runrepeat.com/ranking/rankings-of-running-shoes?page=5
https://runrepeat.com/ranking/rankings-of-running-shoes?page=6
https://runrepeat.com/ranking/rankings-of-running-shoes?page=7
https://runrepeat.com/ranking/rankings-of-running-shoes?page=8
https://runrepeat.com/ranking/rankings-of-running-shoes?page=9
https://runrepeat.com/ranking/rankings-of-running-shoes?page=10
https://runrepeat.com/ranking/rankings-of-running-shoes?page=11
https://runrepeat.com/ranking/rankings-of-running-shoes?page=12
https://runrepeat.com/ranking/rankings-of-running-shoes?page=13
https://runrepeat.com/ranking/rankings-of-running-shoes?page=14
https://runrepeat.com/ranking/rankings-of-running-shoes?page=15
https://runrepeat.com/ranking/rankings-of-running

##     

<a id='t3'></a>
## 3 - Processamento dos dados brutos
- [Sumário](#top) 
    - [Anterior](#t2)
    - [Próximo](#t4)

### Após navegar pelo código fonte da página, é possível identificar algumas *tags* que "guardam" cada informação do site. No nosso caso, recuperamos o link e o nome correspondente a cada tênis disponível.
- Com o auxílio do bs4, os dados extraídos foram salvos em um arquivo JSON.

In [4]:
for page in range(1,77):
    with open("./dados_brutos/{}_{}.html".format(pagina, page), 'r+',encoding="utf-8") as inp:
        page_html = inp.read()

        parsed = bs4.BeautifulSoup(page_html)

        tags = parsed.findAll("a")

        for e in tags:
            if e.has_attr("data-v-45baff66") and e.find("span", itemprop="name") != None:
                link = e['href']
                tenis= e.find("span", itemprop="name").get_text()
                with open("./dados_json/parsed_running_shoes.json", 'a+',encoding="utf-8") as output:
                    data = {"link": link, "name": tenis}
                    output.write("{}\n".format(json.dumps(data)))

##     

<a id='t4'></a>
## 4 - Verificação do resultado
- [Sumário](#top) 
    - [Anterior](#t3)

### Para finalizar este processo, é feita uma verificação dos dados obtidos.
- Para visualização transformamos o arquivo JSON no formato *dataframe* do pandas e verificamos os 7 primeiros e últimos registros;
- Os dados foram coletados com êxito;
- Podemos perceber que existem alguns registros duplicados, porém trataremos isto em outra etapa.

In [5]:
df = pd.read_json("./dados_json/parsed_running_shoes.json", lines=True)

In [6]:
df.head(7)

,link,name
0,/brooks-adrenaline-gts-19?selected_color=422261,Brooks Adrenaline GTS 19
1,/brooks-adrenaline-gts-19?selected_color=422261,Brooks Adrenaline GTS 19
2,/brooks-ghost-12?selected_color=799577,Brooks Ghost 12
3,/brooks-ghost-12?selected_color=799577,Brooks Ghost 12
4,/brooks-glycerin-17?selected_color=642780,Brooks Glycerin 17
5,/brooks-glycerin-17?selected_color=642780,Brooks Glycerin 17
6,/nike-air-zoom-pegasus-36?selected_color=712129,Nike Air Zoom Pegasus 36


In [7]:
df.tail(7)

,link,name
9101,/hoka-one-one-huaka-2,Hoka One One Huaka 2
9102,/salming-race-7,Salming Race 7
9103,/salming-race-7,Salming Race 7
9104,/mizuno-wave-rider-tt,Mizuno Wave Rider TT
9105,/mizuno-wave-rider-tt,Mizuno Wave Rider TT
9106,/salming-trail-t4?selected_color=852528,Salming Trail T4
9107,/salming-trail-t4?selected_color=852528,Salming Trail T4
